In [1]:
import json
import os

import numpy as np
import polars as pol
from sklearn.model_selection import train_test_split

In [2]:
df = pol.read_csv("../data/vodafone_data_short_head.csv")

In [3]:
df.head()

Incident_No,Description,Assigned_Group,Incident_Type,Production_Category,Operational_Category,n_words,clean_text,Assigned_Group_fixed
str,str,str,str,str,str,i64,str,str
"""INC00000776724…","""New Campaign |…","""CM Support""","""User Service R…","""Business Appli…","""Critical Appli…",32,"""new campaign …","""cm support"""
"""INC00000776689…","""CRM Applicatio…","""War Room""","""User Service R…","""Business Appli…","""Issue_Applicat…",69,"""crm applicatio…","""war room"""
"""INC00000776602…","""vpn id not wor…","""Dispatcher""","""User Service R…","""Software_Opera…","""Issue_Access_F…",62,"""vpn id not wor…","""dispatcher"""
"""INC00000776570…","""Mview Refresh …","""SSO-DBA-Dispat…","""User Service R…","""Business Appli…","""Central Applic…",48,"""mview refresh …","""sso-dba"""
"""INC00000776570…","""OS login issue…","""Dispatcher""","""User Service R…","""Software_Opera…","""Hardware_Acces…",30,"""os login issue…","""dispatcher"""


In [4]:
label_map = {
    k: idx for idx, k in enumerate(df["Assigned_Group_fixed"].unique().to_list())
}
label_map

{'iam provisioning': 0,
 'sso-eai-reccpe': 1,
 'l2 smart app': 2,
 'iam-sarm': 3,
 'war room': 4,
 'project everest bpa': 5,
 'evo tech scp': 6,
 'ebpp': 7,
 'recc': 8,
 'sdp-apoena amp': 9,
 'bi ': 10,
 'l2-cti support': 11,
 'disha crm-crm-pune-metropolitan': 12,
 'l2-vtopup': 13,
 'network': 14,
 'ro': 15,
 'sso-apm-wintel': 16,
 'user id mgmt': 17,
 'mdm-l1 support': 18,
 'amdocs_': 19,
 'siebelcrm_hyper care_l1 ops_techm': 20,
 'vodafone cloud l1': 21,
 'central application-npg': 22,
 'cit-upss-upss-support': 23,
 'l2-nsms support': 24,
 'service desk-fcr': 25,
 'mediation': 26,
 'enterprise crm operation support': 27,
 'billing configuration': 28,
 'l2-simex support': 29,
 'l2-crm ops support': 30,
 'sam operation ops': 31,
 'ra mediation-pune-metropolitan': 32,
 'sso-remedy': 33,
 'its-ssk payment': 34,
 'eshop': 35,
 'amdocs_bpm_am': 36,
 'l1-crm postpaid support': 37,
 'cit-cpos-cpossupport': 38,
 'cm support': 39,
 'l2 dhruv support': 40,
 'sso-opssupport': 41,
 'l2-ipops_cms

In [5]:
df = df.with_columns(
    (pol.col("Assigned_Group_fixed").map_dict(label_map)).alias("label")
)
df.head()

Incident_No,Description,Assigned_Group,Incident_Type,Production_Category,Operational_Category,n_words,clean_text,Assigned_Group_fixed,label
str,str,str,str,str,str,i64,str,str,i64
"""INC00000776724…","""New Campaign |…","""CM Support""","""User Service R…","""Business Appli…","""Critical Appli…",32,"""new campaign …","""cm support""",39
"""INC00000776689…","""CRM Applicatio…","""War Room""","""User Service R…","""Business Appli…","""Issue_Applicat…",69,"""crm applicatio…","""war room""",4
"""INC00000776602…","""vpn id not wor…","""Dispatcher""","""User Service R…","""Software_Opera…","""Issue_Access_F…",62,"""vpn id not wor…","""dispatcher""",45
"""INC00000776570…","""Mview Refresh …","""SSO-DBA-Dispat…","""User Service R…","""Business Appli…","""Central Applic…",48,"""mview refresh …","""sso-dba""",53
"""INC00000776570…","""OS login issue…","""Dispatcher""","""User Service R…","""Software_Opera…","""Hardware_Acces…",30,"""os login issue…","""dispatcher""",45


In [6]:
df_train, df_test = train_test_split(
    df, test_size=0.2, random_state=32, shuffle=True, stratify=df.select("label")
)

df_val, df_test = train_test_split(df_test, test_size=0.5, shuffle=False)

print(f"Shape of the training data: {df_train.shape}")
print(f"Shape of the validation data: {df_val.shape}")
print(f"Shape of the test data: {df_test.shape}")

Shape of the training data: (35160, 10)
Shape of the validation data: (4395, 10)
Shape of the test data: (4395, 10)


In [7]:
if not os.path.exists("../data/voda_idea_data_splits/"):
    os.mkdir("../data/voda_idea_data_splits/")

In [8]:
df_train.write_csv("../data/voda_idea_data_splits/train.csv")

In [9]:
df_val.write_csv("../data/voda_idea_data_splits/val.csv")

In [10]:
df_test.write_csv("../data/voda_idea_data_splits/test.csv")

In [11]:
with open("../data/voda_idea_data_splits/label_map.json", "w") as fp:
    json.dump(label_map, fp, indent=4)